This code is used to test the saved model

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
from gymnasium.wrappers import GrayScaleObservation
from gymnasium.wrappers import ResizeObservation
from gymnasium.wrappers import FrameStack, RecordVideo
from tensorflow import keras
from collections import deque

#np.random.seed(42)

name = "Freeway"
model_path = "./Freeway-v5/best_run/policy/"
model = tf.keras.models.load_model(model_path)

In [ ]:
env = gym.make(f'ALE/{name}-v5',
                frameskip=4,
                render_mode="human",
                max_episode_steps=18000/4)

#env = RecordVideo(env, f"./videos/{name}")
env = GrayScaleObservation(env)
env = ResizeObservation(env, 84)
env = FrameStack(env,4)

In [ ]:
def play_one_step(env, state):
    state = np.float32(state) / 255
    actions_probs, _, _ = model.predict(state, verbose=0)
    action = np.random.choice(np.arange(env.action_space.n), p=np.asarray(actions_probs.squeeze()))

    next_state, reward, term, trunc, _,  = env.step(action)
    #record_env.step(action)
    next_state = np.expand_dims(next_state, axis=0)
    next_state = np.expand_dims(next_state, axis=-1)
    
    return next_state, reward, term, trunc

In [ ]:
episodes = 50
mean_rwd = 0 

state,_ = env.reset()
#record_env.reset()
#record_env.start_video_recorder()
state = np.expand_dims(state, axis=0)
state = np.expand_dims(state, axis=-1)


In [ ]:
count = 1
while True:
    state, reward, term, trunc = play_one_step(env, state)
    mean_rwd = mean_rwd + 1/count * (reward - mean_rwd)
    count += 1
    #env.render()
    if term or trunc:
        break
#record_env.close_video_recorder()
env.close()